In [1]:
import os
import torch
import faiss
import logging
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from Libraries import A0_MyUtils as A0, A1_TextProcess as A1, A2_PdfProcess as A2
from Libraries import B1_ExtractData as B1, B2_MergeData as B2, B3_GetStructures as B3
from Libraries import B4_ChunkMaster as B4, B5_ChunkFlex as B5, B6_ChunkFixed as B6
from Libraries import C1_CreateSchema as C1, C2_Embedding as C2, C3_CheckStruct as C3
from Libraries import D0_FaissConvert as D0, D1_Search as D1, D2_Rerank as D2, D3_Respond as D3
from Config import Widgets, Configs, ModelLoader as ML

In [2]:
widgets_list = Widgets.create_name_form()

Dropdown(description='Model: ', index=1, layout=Layout(width='90%'), options=('Auto Model', 'Sentence Transfor…

Dropdown(description='Embedder: ', index=2, layout=Layout(width='90%'), options=('vinai/phobert-base', 'keepit…

Dropdown(description='Searcher: ', index=1, layout=Layout(width='90%'), options=('faiss.IndexHNSWFlat', 'faiss…

Dropdown(description='Reranker: ', layout=Layout(width='90%'), options=('BAAI/bge-reranker-base',), value='BAA…

Dropdown(description='Response: ', layout=Layout(width='90%'), options=('gemini-2.0-flash-exp', 'vinai/PhoGPT-…

Dropdown(description='API Key:', index=4, layout=Layout(width='90%'), options=('AIzaSyDaHS-8h6GJkyVPhoX4svvYeB…

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [4]:
config = Configs.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
extracted_path = config["extracted_path"]
merged_path = config["merged_path"]
struct_path = config["struct_path"]
chunks_base = config["chunks_base"]
chunks_segment = config["chunks_segment"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
map_data_path = config["map_data_path"]
meta_path = config["meta_path"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]

SEARCH_ENGINE = faiss.IndexFlatIP

Model   : Sentence Transformer
Type    : Data
Embedder: VoVanPhuc/sup-SimCSE-VietNamese-phobert-base
Searcher: faiss.IndexFlatIP
Reranker: BAAI/bge-reranker-base
Responer: gemini-2.0-flash-exp
Data Key: contents
Embe Key: embeddings
File    : HNMU_Regulations
Dcment  : ../Docs/HNMU_Regulations.pdf
Extract : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Extracted.json
Merge   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Merged.json
Struct  : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Struct.json
Chunked : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Chunks.json
Segment : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Segment.json
Torch   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings.pt
Faiss   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings.faiss
Mapping : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings_mapping.json
Map Data: ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings_map_data.json
Meta    : ../D

## Prepare

### Fixed

In [5]:
# loaded model
cached_path = "../Models"

# assets
assets = "../Assets/"
exceptions_path = f"{assets}ex.exceptions.json"
markers_path = f"{assets}ex.markers.json"
status_path = f"{assets}ex.status.json"

### Device

In [6]:
ML.CudaCheck()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if SWITCH == "Auto Model":
    if os.path.exists(cached_path):
        tokenizer, model = ML.load_auto_model(cached_path, device)
        print(f"ℹ️ Auto Model: {cached_path}")
        if model is None:
            tokenizer, model = ML.load_auto_model(EMBEDD_MODEL, device)
    else:
        print(f"ℹ️ Auto Model: {EMBEDD_MODEL}")
        tokenizer, model = ML.load_auto_model(EMBEDD_MODEL, device)

elif SWITCH == "Sentence Transformer":
    if os.path.exists(cached_path):
        model = ML.load_sentence_model(cached_path, device)
        print(f"ℹ️ Sentece Transformer: {cached_path}")

        if model is None:
            model = ML.load_sentence_model(EMBEDD_MODEL, device)
    else:
        print(f"ℹ️ Sentece Transformer: {EMBEDD_MODEL}")
        model = ML.load_sentence_model(EMBEDD_MODEL, device)

print(f"✅ Using: {device}")

CUDA supported: True
Number of GPUs: 1
Current GPU name: NVIDIA GeForce RTX 2050
CUDA device capability: (8, 6)
CUDA version (PyTorch): 12.1
cuDNN version: 90100
ℹ️ Sentece Transformer: VoVanPhuc/sup-SimCSE-VietNamese-phobert-base


c:\Users\Orias.ASUS\miniconda3\envs\master\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Using: cuda


### Import Classes

In [7]:
dataExtractor = B1.B1Extractor(
    exceptions_path, 
    markers_path, 
    status_path, 
    proper_name_min_count=10
)

In [8]:
structAnalyzer = B3.StructureAnalyzer(
    merged_path = merged_path,
    verbose=True
)

In [9]:
chunkBuilder = B4.ChunkBuilder(
    struct_path=struct_path,
    merged_path=merged_path,
)

In [10]:
schemaEx = C1.JSONSchemaExtractor(
    list_policy="first", 
    verbose=True
)

In [11]:
Embedding = C2.JSONEmbedding(
    model=model,
    device="cuda:0",
    batch_size=32,
    show_progress=False,
    flatten_mode="split"
)

In [12]:
FaissConverter = D0.Torch2FaissConverter(
    schema_ex_path=schema_ex_path,
    torch_path=torch_path,
    faiss_path=faiss_path,
    mapping_path=mapping_path,
    map_data_path=map_data_path,
    keep_last=2,
    nlist=100,
    mode=EMBE_KEY,
    use_pickle=False
)

2025-09-29 15:56:25,222 - INFO - Chỉ giữ 2 field cuối trong schema: {'Article', 'Content'}


### Extract Data

In [13]:
def extractRun():
    extracted_data = dataExtractor.extract(dcmt_path)
    A0.write_json(extracted_data, extracted_path, indent=1)

    merged_data = B2.mergeLinesToParagraphs(extracted_data)
    A0.write_json(merged_data, merged_path, indent=1)

### Get Struct

In [14]:
def structRun():
    markers = structAnalyzer.extract_markers()

    structures = structAnalyzer.build_structures(markers)
    print(A0.jsonl_convert(structures))

    dedup = structAnalyzer.deduplicate(structures)
    print(A0.jsonl_convert(dedup))

    top = structAnalyzer.select_top(dedup)
    topext = structAnalyzer.extend_top(top, dedup)
    print(A0.json_convert(topext, pretty=True))

    A0.write_json(topext, struct_path, indent=2)

### Chunks

In [15]:
def chunkRun():
    chunks = chunkBuilder.build()
    A0.write_json(chunks, chunks_base, indent=2)

    filtered = [item for item in chunks if item.get("Level 1", "").strip()]
    for i, item in enumerate(filtered, start=1):
        item["Index"] = i

    A0.write_json(filtered, chunks_segment, indent=2)

### Schema Extract

In [16]:
def schemaRun():
    if os.path.exists(chunks_segment):
        schemaEx.schemaRun(chunks_segment, schema_path=schema_ex_path)
        chunksSchema = A0.read_json(schema_ex_path)
        print(chunksSchema)
    else:
        print(f"{chunks_segment} does not exist")

### Embedding

In [ ]:
def embeddingRun():
    if os.path.exists(chunks_segment):
        Embedding.embeddingRun(
            json_path = chunks_segment,
            schema_path = schema_ex_path,
            torch_path = torch_path,
            data_key = DATA_KEY,
            embe_key = EMBE_KEY,
            skip_if_exists = False,
        )
        
        C3.print_json(DATA_KEY, EMBE_KEY, torch_path)
    
    else:
        print(f"{chunks_segment} does not exist")

### Run

In [ ]:
if FILE_TYPE == "Data":
    extractRun()
    structRun()
    chunkRun()
    
schemaRun()
embeddingRun()
FaissConverter.convert()


Checking fields and their types:
New: Index           item 1: Type = number
New: Level 1         item 1: Type = string
New: Level 2         item 1: Type = string
New: Article         item 1: Type = string
New: Content         item 1: Type = array
Generated schema:
{
  "Index": "number",
  "Level 1": "string",
  "Level 2": "string",
  "Article": "string",
  "Content": "array"
}
{'Index': 'number', 'Level 1': 'string', 'Level 2': 'string', 'Article': 'string', 'Content': 'array'}
Đã lưu embedding vào: ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings.pt
[contents] Tổng số mục: 468 — In mục index=0
{
  "Index": 1,
  "Level 1": "Chương I NHỮNG QUY ĐỊNH CHUNG",
  "Level 2": "Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng",
  "Article": "1. Quy chế này quy định chung về tổ chức và quản lý đào tạo trình độ đại học tại trường Đại học Thủ đô Hà Nội bao gồm: Chương trình đào tạo và thời gian học tập; hình thức và phương thức tổ chức đào t…",
  "Content": {
    "__vector_len__": 0,
    

2025-09-29 15:56:45,981 - INFO - Kiểu dữ liệu: <class 'dict'>
2025-09-29 15:56:45,982 - INFO - Số lượng khóa cấp cao nhất: 2
2025-09-29 15:56:46,985 - INFO - Khóa: contents, Kiểu giá trị: <class 'list'>, Giá trị mẫu: [{'Index': 1, 'Level 1': 'Chương I NHỮNG QUY ĐỊNH CHUNG', 'Level 2': 'Điều 1. Phạm vi điều chỉnh và ...
2025-09-29 15:56:47,996 - INFO - Khóa: embeddings, Kiểu giá trị: <class 'list'>, Giá trị mẫu: [{'Level 1 Embedding': [-0.2727910876274109, -0.008815516717731953, -0.12173040211200714, -0.7325150...
2025-09-29 15:56:48,009 - INFO - Đang tải file .pt: ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings.pt
2025-09-29 15:56:48,188 - INFO - Đang trích xuất embedding và dữ liệu...
2025-09-29 15:56:48,239 - INFO - Tìm thấy 1056 embedding.
2025-09-29 15:56:48,240 - INFO - Đang tạo chỉ mục FAISS...
2025-09-29 15:56:48,244 - INFO - Đang thêm embedding vào chỉ mục...
2025-09-29 15:56:48,247 - INFO - Đang lưu ánh xạ khóa vào ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embed

## RAG

### Funcs

In [19]:
def searchRun(user_question):
    preliminary_results = D1.search_faiss_index(
        query = user_question,
        embedd_model = EMBEDD_MODEL,
        faiss_path = faiss_path,
        mapping_path = mapping_path,
        mapping_data = map_data_path,
        device = device,
        k = 2,
        min_score = 5,
        batches = False,
    )
    return preliminary_results

def rerankRun(user_question, preliminary_results):
    reranked_results = D2.rerank_results(
        query = user_question,
        results = preliminary_results,
        reranker_model = RERANK_MODEL,
        device = device,
        k = 5,
        batches = False,
    )
    context = '\n\n'.join(item['text'] for item in reranked_results)
    return reranked_results, context

def respondRun(user_question, prompt, context="", doc=False):
    response = D3.respond_naturally(
        user_question = user_question,
        context = context,
        prompt = prompt,
        responser_model = RESPON_MODEL,
        score_threshold = 0.85,
        max_results = 3,
        doc = doc,
        gemini_api_key = API_KEY,
    )
    return response

### Main

In [20]:
prompt_type = "Docs"

queries = [
    "Quy chế này quy định những gì và áp dụng cho đối tượng nào",
    "Sinh viên có thể được thi lại bao nhiêu lần?",
]

def chatRun():

    prompt_path = f"Prompts/{prompt_type}_Prompt.txt"
    with open(prompt_path, "r", encoding="utf-8") as f: prompt = f.read()

    print("<< Enter 'exit', 'quit', 'escape', 'bye' or Press ESC to exit >>")
    print("Chatbot: Hello there! I'm here to help you!\n\n")

    query = -1
    while True:
        try:
            query += 1
            if query >= len(queries):
                user_input = "exit"
            else:
                user_input = queries[query]

            # user_input = input("You: ")

            user_question = A0.preprocess_text(user_input)
            if user_input.strip().lower() in ["exit", "quit", "escape", "bye", ""]:
                print("Chatbot: Goodbye!")
                break

            print(f"Query: {user_question}")

            # Bước 1: Search
            preliminary_results = searchRun(user_question)
            
            # Bước 2: Rerank
            if preliminary_results:
                print(preliminary_results)
                reranked_results, context = rerankRun(user_question, preliminary_results)
            else:
                print("Không tìm thấy thông tin!")

            # Bước 3: Generate Response
            if (reranked_results):
                print(reranked_results)
                print(f"\n Context:\n {context}")
                response = respondRun(user_question, prompt, context="", doc=True)
            else:
                print("Rerank thất bại!")

            # Bước 4: Print Response
            if response:
                print(f"\nYou: {user_question}")
                print(f"Chatbot: {response}\n\n")
            else:
                print("LLM không phản hồi!")

            print("=" * 200)
            print("\n\n")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break

In [21]:
# chatRun()